## Zadania: Ewaluacja - porównanie łańcuchów tekstowych

**Temat:** BLEU, ROUGE, METEOR.

## Zadanie 1 - BLEU

In [ ]:
# %pip install nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

reference = ["to","jest","krótka","odpowiedź"]      # TODO: ewentualnie zmień
candidate = ["to","jest","krótka","odpowiedz"]      # TODO: ewentualnie zmień

weights = (1.0, 0.0, 0.0, 0.0)                      # TODO: BLEU-1
score = sentence_bleu([reference], candidate, weights=weights, smoothing_function=SmoothingFunction().method1)
print("BLEU-1:", round(score, 4))
assert 0.0 <= score <= 1.0


## Zadanie 2 - ROUGE-L (rouge-score)

In [ ]:
# %pip install rouge-score
from rouge_score import rouge_scorer
ref = "To jest prosty przykład metryki ROUGE."
pred = "To jest przykład bardzo prostego ROUGE."
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)  # TODO: zmień jeśli chcesz
scores = scorer.score(ref, pred)
print("ROUGE-L F1:", round(scores["rougeL"].fmeasure, 4))
assert 0.0 <= scores["rougeL"].fmeasure <= 1.0


## Zadanie 3 - METEOR

In [ ]:
# %pip install nltk
from nltk.translate.meteor_score import meteor_score
ref = "Szybki brązowy lis przeskakuje nad leniwym psem"
cand = "Brązowy lis szybko skacze nad leniwym psem"
meteor = meteor_score([ref], cand)  # TODO: uzupełnij poprawnie listę referencji
print("METEOR:", round(meteor, 4))
assert 0.0 <= meteor <= 1.0
